In [ ]:
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt
import optuna

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from joblib import Parallel, delayed

def load_tracks(filepath):
    tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])
    _expand_string_lists(tracks)
    _convert_date_columns(tracks)
    _convert_subsets(tracks)
    _convert_to_category(tracks)
    return tracks

def _expand_string_lists(tracks_df):
    columns_as_lists = [
        ('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
        ('track', 'genres'), ('track', 'genres_all')
    ]
    for col in columns_as_lists:
        tracks_df[col] = tracks_df[col].map(ast.literal_eval)

def _convert_date_columns(tracks_df):
    date_columns = [
        ('track', 'date_created'), ('track', 'date_recorded'),
        ('album', 'date_created'), ('album', 'date_released'),
        ('artist', 'date_created'), ('artist', 'active_year_begin'),
        ('artist', 'active_year_end')
    ]
    for col in date_columns:
        tracks_df[col] = pd.to_datetime(tracks_df[col])

def _convert_subsets(tracks_df):
    from pandas.api.types import CategoricalDtype
    subsets = ('small', 'medium', 'large')
    tracks_df['set', 'subset'] = tracks_df['set', 'subset'].astype(
        CategoricalDtype(categories=subsets, ordered=True)
    )

def _convert_to_category(tracks_df):
    cat_cols = [
        ('track', 'genre_top'), ('track', 'license'),
        ('album', 'type'), ('album', 'information'),
        ('artist', 'bio')
    ]
    for col in cat_cols:
        tracks_df[col] = tracks_df[col].astype('category')

def load_features(filepath):
    df = pd.read_csv(filepath, index_col=0, header=[0, 1, 2])
    new_cols = [f"{lvl0}_{lvl1}_{lvl2}" for lvl0, lvl1, lvl2 in df.columns.to_list()]
    df.columns = new_cols
    return df


base_dir = "C:/Users/Climclaff/Downloads/fma/fma_metadata"
tracks_path = os.path.join(base_dir, "tracks.csv")
features_path = os.path.join(base_dir, "features.csv")

tracks = load_tracks(tracks_path)
features = load_features(features_path)

small_mask = (tracks['set', 'subset'] == 'small')
excluded_genres = ["Pop", "Folk"]
genre_mask = ~tracks['track', 'genre_top'].isin(excluded_genres)
filtered_mask = small_mask & genre_mask

all_columns = features.columns
feature_groups = {
    "chroma_cens":        [c for c in all_columns if "chroma_cens" in c.lower()],
    "chroma_cqt":         [c for c in all_columns if "chroma_cqt" in c.lower()],
    "chroma_stft":        [c for c in all_columns if "chroma_stft" in c.lower()],
    "mfcc":               [c for c in all_columns if "mfcc" in c.lower()],
    "rmse":               [c for c in all_columns if "rmse" in c.lower()],
    "spectral_bandwidth": [c for c in all_columns if "spectral_bandwidth" in c.lower()],
    "spectral_centroid":  [c for c in all_columns if "spectral_centroid" in c.lower()],
    "spectral_contrast":  [c for c in all_columns if "spectral_contrast" in c.lower()],
    "spectral_rolloff":   [c for c in all_columns if "spectral_rolloff" in c.lower()],
    "tonnetz":            [c for c in all_columns if "tonnetz" in c.lower()],
    "zcr":                [c for c in all_columns if "zcr" in c.lower()]
}
best_feature_groups = ('chroma_cqt', 'chroma_stft', 'mfcc', 'rmse',
                       'spectral_centroid', 'spectral_contrast', 'tonnetz', 'zcr')
selected_cols = []
for group in best_feature_groups:
    selected_cols += feature_groups[group]

X = features.loc[filtered_mask, selected_cols].copy()
y = tracks.loc[filtered_mask, ('track', 'genre_top')].copy()
X_all = X.values
y_all = y.values

def create_svc():
    class_weights = {
        "Electronic": 1.6,
        "Experimental": 1.8,
        "Hip-Hop": 1.0,
        "Instrumental": 1.2,
        "International": 1.0,
        "Rock": 1.0
    }
    svc = SVC(
        C=3.750298340519889,
        kernel='rbf',
        gamma='auto',
        class_weight=class_weights,
        probability=True,
        random_state=213
    )
    return make_pipeline(StandardScaler(), svc)

def create_bagging():
    best_params_bagging = {
        'n_estimators': 264,
        'max_samples': 0.892487331464192,
        'max_features': 0.67051206388554,
        'bootstrap': False,
        'max_depth': 11,
        'min_samples_split': 12
    }
    base_tree = DecisionTreeClassifier(
        max_depth=best_params_bagging['max_depth'],
        min_samples_split=best_params_bagging['min_samples_split'],
        random_state=213
    )
    return BaggingClassifier(
        estimator=base_tree,
        n_estimators=best_params_bagging['n_estimators'],
        max_samples=best_params_bagging['max_samples'],
        max_features=best_params_bagging['max_features'],
        bootstrap=best_params_bagging['bootstrap'],
        n_jobs=-1,
        random_state=42
    )

def create_adaboost():
    best_params_adaboost = {
        'n_estimators': 349,
        'learning_rate': 0.11314525855173757,
        'max_depth': 8,
        'min_samples_split': 21,
        'min_samples_leaf': 20,
        'criterion': 'gini',
        'class_weight': 'balanced'
    }
    optimized_tree = DecisionTreeClassifier(
        max_depth=best_params_adaboost['max_depth'],
        min_samples_split=best_params_adaboost['min_samples_split'],
        min_samples_leaf=best_params_adaboost['min_samples_leaf'],
        criterion=best_params_adaboost['criterion'],
        class_weight=best_params_adaboost['class_weight'],
        random_state=213
    )
    return AdaBoostClassifier(
        estimator=optimized_tree,
        n_estimators=best_params_adaboost['n_estimators'],
        learning_rate=best_params_adaboost['learning_rate'],
        random_state=42
    )

def create_random_forest():
    best_params_rf = {
        'n_estimators': 398,
        'max_samples': 0.8910775456540504,
        'max_depth': 19,
        'min_samples_split': 7,
        'min_samples_leaf': 3,
        'max_features': 0.6650498730427247,
        'class_weight': 'balanced',
        'criterion': 'gini'
    }
    return RandomForestClassifier(
        n_estimators=best_params_rf['n_estimators'],
        max_samples=best_params_rf['max_samples'],
        max_depth=best_params_rf['max_depth'],
        min_samples_split=best_params_rf['min_samples_split'],
        min_samples_leaf=best_params_rf['min_samples_leaf'],
        max_features=best_params_rf['max_features'],
        class_weight=best_params_rf['class_weight'],
        criterion=best_params_rf['criterion'],
        n_jobs=-1,
        random_state=213
    )

base_model_fns_stack = [create_svc, create_bagging, create_adaboost, create_random_forest]

def get_fold_oof(constructor_fn, X, y, train_idx, valid_idx):
    model = constructor_fn()
    model.fit(X[train_idx], y[train_idx])
    return valid_idx, model.predict_proba(X[valid_idx])

def get_oof_probs_parallel(X, y, base_model_constructors, n_splits=3, shuffle=True, random_state=123):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    n_classes = len(np.unique(y))
    n_models = len(base_model_constructors)
    oof_prob_matrix = np.zeros((X.shape[0], n_models * n_classes), dtype=np.float64)
    
    for m_idx, constructor_fn in enumerate(base_model_constructors):
        results = Parallel(n_jobs=-1)(
            delayed(get_fold_oof)(constructor_fn, X, y, train_idx, valid_idx)
            for train_idx, valid_idx in skf.split(X, y)
        )
        for valid_idx, prob_val in results:
            start_col = m_idx * n_classes
            end_col = start_col + n_classes
            oof_prob_matrix[valid_idx, start_col:end_col] = prob_val
    return oof_prob_matrix

def train_base_models_full(X, y, base_model_constructors):
    models = []
    for fn in base_model_constructors:
        model = fn()
        model.fit(X, y)
        models.append(model)
    return models

def get_base_model_probs(models, X):
    n_samples = X.shape[0]
    n_models = len(models)
    n_classes = models[0].predict_proba(X[:1]).shape[1]
    prob_matrix = np.zeros((n_samples, n_models * n_classes), dtype=np.float64)
    for i, m in enumerate(models):
        proba = m.predict_proba(X)
        start_col = i * n_classes
        end_col = start_col + n_classes
        prob_matrix[:, start_col:end_col] = proba
    return prob_matrix



sss_opt = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx_opt, val_idx_opt = next(sss_opt.split(X_all, y_all))

X_train_opt, X_val_opt = X_all[train_idx_opt], X_all[val_idx_opt]
y_train_opt, y_val_opt = y_all[train_idx_opt], y_all[val_idx_opt]

In [ ]:
print("\nGenerating OOF features for optimization split...")
oof_probs_train_opt = get_oof_probs_parallel(X_train_opt, y_train_opt, base_model_fns_stack, n_splits=3)

print("Training base models on full optimization training set...")
final_models_opt = train_base_models_full(X_train_opt, y_train_opt, base_model_fns_stack)

print("Generating validation predictions...")
val_probs_opt = get_base_model_probs(final_models_opt, X_val_opt)

In [ ]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def objective_4model(trial):
    """Objective function for 4-model stacking."""
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    
    lr_meta = LogisticRegression(
        C=C,
        penalty='l2',
        solver='lbfgs',
        max_iter=1000,
        class_weight=None,
        random_state=999
    )
    
    lr_meta.fit(oof_probs_train_opt, y_train_opt)
    val_preds = lr_meta.predict(val_probs_opt)
    return accuracy_score(y_val_opt, val_preds)

study_4model = optuna.create_study(direction='maximize')
study_4model.optimize(objective_4model, n_trials=1000)

BEST_LR_META_PARAMS_4MODELS = study_4model.best_params
print(f"\nOptimized 4-model LR Meta Params: {BEST_LR_META_PARAMS_4MODELS}")
print(f"Best validation accuracy (4-model stack): {study_4model.best_value:.4f}")

base_model_fns_stack_2 = [create_svc, create_adaboost]

print("\nGenerating OOF features for 2-model (SVC + AdaBoost) on optimization split...")
oof_probs_train_opt_2 = get_oof_probs_parallel(
    X_train_opt, y_train_opt,
    base_model_fns_stack_2,
    n_splits=3, shuffle=True, random_state=123
)

print("Training base models (2-model) on full optimization training set...")
final_models_opt_2 = train_base_models_full(
    X_train_opt, y_train_opt,
    base_model_fns_stack_2
)

print("Generating validation predictions for 2-model stack...")
val_probs_opt_2 = get_base_model_probs(final_models_opt_2, X_val_opt)

def objective_2model(trial):
    """Objective function for 2-model stacking (SVC + AdaBoost)."""
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    
    lr_meta_2 = LogisticRegression(
        C=C,
        penalty='l2',
        solver='lbfgs',
        max_iter=1000,
        class_weight=None,
        random_state=999
    )
    
    lr_meta_2.fit(oof_probs_train_opt_2, y_train_opt)
    val_preds_2 = lr_meta_2.predict(val_probs_opt_2)
    return accuracy_score(y_val_opt, val_preds_2)

study_2model = optuna.create_study(direction='maximize')
study_2model.optimize(objective_2model, n_trials=1000)

BEST_LR_META_PARAMS_2MODELS = study_2model.best_params
print(f"\nOptimized 2-model (SVC+AdaBoost) LR Meta Params: {BEST_LR_META_PARAMS_2MODELS}")
print(f"Best validation accuracy (2-model stack): {study_2model.best_value:.4f}")

In [ ]:
import os, joblib, numpy as np
from sklearn.metrics         import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.tree            import DecisionTreeClassifier
from sklearn.linear_model    import LogisticRegression

SAVE_DIR = r"E:\ML_Models"
os.makedirs(SAVE_DIR, exist_ok=True)

# --------------------------------------------------
# helpers
# --------------------------------------------------
def get_classification_metrics(y_true, y_pred):
    return {
        'accuracy' : accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall'   : recall_score(y_true, y_pred, average='weighted'),
        'f1'       : f1_score(y_true, y_pred, average='weighted')
    }

def evaluate_standalone_models(X_train, y_train, X_val, y_val):
    metrics = {}

    dt_model = DecisionTreeClassifier(max_depth=11,
                                      min_samples_split=12,
                                      random_state=213)
    dt_model.fit(X_train, y_train)
    metrics['Decision Tree'] = get_classification_metrics(y_val, dt_model.predict(X_val))

    bagging_model = create_bagging();  bagging_model.fit(X_train, y_train)
    metrics['Bagging'] = get_classification_metrics(y_val, bagging_model.predict(X_val))

    adaboost_model = create_adaboost(); adaboost_model.fit(X_train, y_train)
    metrics['AdaBoost'] = get_classification_metrics(y_val, adaboost_model.predict(X_val))

    rf_model = create_random_forest(); rf_model.fit(X_train, y_train)
    metrics['Random Forest'] = get_classification_metrics(y_val, rf_model.predict(X_val))

    return metrics

def evaluate_fold(Xtr, ytr, Xval, yval):
    results = {}
    results.update(evaluate_standalone_models(Xtr, ytr, Xval, yval))

    svc_model = create_svc(); svc_model.fit(Xtr, ytr)
    results['SVC'] = get_classification_metrics(yval, svc_model.predict(Xval))

    # 4-model stack
    oof4 = get_oof_probs_parallel(Xtr, ytr, base_model_fns_stack, n_splits=3)
    lr4  = LogisticRegression(**BEST_LR_META_PARAMS_4MODELS, solver='lbfgs',
                              max_iter=1000, random_state=999)
    lr4.fit(oof4, ytr)
    base4 = train_base_models_full(Xtr, ytr, base_model_fns_stack)
    val4  = get_base_model_probs(base4, Xval)
    results['Stacking (4-model)'] = get_classification_metrics(yval, lr4.predict(val4))

    # 2-model stack (SVC+AdaBoost)
    two_fns = [create_svc, create_adaboost]
    oof2 = get_oof_probs_parallel(Xtr, ytr, two_fns, n_splits=3)
    lr2  = LogisticRegression(**BEST_LR_META_PARAMS_2MODELS, solver='lbfgs',
                              max_iter=1000, random_state=999)
    lr2.fit(oof2, ytr)
    base2 = train_base_models_full(Xtr, ytr, two_fns)
    val2  = get_base_model_probs(base2, Xval)
    results['Stacking (2-model)'] = get_classification_metrics(yval, lr2.predict(val2))

    return results

all_metrics = {k: [] for k in ('Decision Tree','Bagging','AdaBoost',
                               'Random Forest','SVC',
                               'Stacking (4-model)','Stacking (2-model)')}

sss = StratifiedShuffleSplit(n_splits=5, test_size=.2, random_state=2023)
for i, (tr, val) in enumerate(sss.split(X_all, y_all), 1):
    res = evaluate_fold(X_all[tr], y_all[tr], X_all[val], y_all[val])
    print(f"\nFold {i} Results:")
    for name, m in res.items():
        print(f"{name:<22} | Acc={m['accuracy']:.4f}, Prec={m['precision']:.4f}, "
              f"Rec={m['recall']:.4f}, F1={m['f1']:.4f}")
        all_metrics[name].append(m)

print("\n=== Final Evaluation Results (mean ± std over folds) ===")
for name, mlist in all_metrics.items():
    a = np.array([m['accuracy']  for m in mlist])
    p = np.array([m['precision'] for m in mlist])
    r = np.array([m['recall']    for m in mlist])
    f = np.array([m['f1']        for m in mlist])
    print(f"{name:<22} | Acc={a.mean():.4f}±{a.std():.4f}, "
          f"Prec={p.mean():.4f}±{p.std():.4f}, "
          f"Rec={r.mean():.4f}±{r.std():.4f}, "
          f"F1={f.mean():.4f}±{f.std():.4f}")


final_models = {}

# stand-alone
final_models['Bagging']       = create_bagging().fit(X_all, y_all)
final_models['AdaBoost']      = create_adaboost().fit(X_all, y_all)
final_models['RandomForest']  = create_random_forest().fit(X_all, y_all)
final_models['SVC']           = create_svc().fit(X_all, y_all)

# 4-model stack
oof_full_4 = get_oof_probs_parallel(X_all, y_all, base_model_fns_stack, n_splits=5)
meta4      = LogisticRegression(**BEST_LR_META_PARAMS_4MODELS, solver='lbfgs',
                                max_iter=1000, random_state=999)
meta4.fit(oof_full_4, y_all)
base_full_4 = train_base_models_full(X_all, y_all, base_model_fns_stack)
final_models['Stack_4_model'] = {"meta": meta4, "base": base_full_4}

# 2-model stack
two_fns     = [create_svc, create_adaboost]
oof_full_2  = get_oof_probs_parallel(X_all, y_all, two_fns, n_splits=5)
meta2       = LogisticRegression(**BEST_LR_META_PARAMS_2MODELS, solver='lbfgs',
                                 max_iter=1000, random_state=999)
meta2.fit(oof_full_2, y_all)
base_full_2 = train_base_models_full(X_all, y_all, two_fns)
final_models['Stack_2_model'] = {"meta": meta2, "base": base_full_2}

for name, model in final_models.items():
    joblib.dump(model, os.path.join(SAVE_DIR, f"{name}.pkl"))

print(f"\nФінальні моделі збережено у: {SAVE_DIR}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ----------------------------------------------------------
# 1. Дані з підсумкової таблиці CV-оцінювання
# ----------------------------------------------------------
model_names = [
    "Decision Tree",
    "Bagging",
    "AdaBoost",
    "Random Forest",
    "SVC",
    "Stacking (4-model)",
    "Stacking (2-model)",
]

# середні значення
acc_means = np.array([
    0.4728,   # Decision Tree
    0.6228,   # Bagging
    0.6775,   # AdaBoost
    0.6338,   # Random Forest
    0.7035,   # SVC
    0.7152,   # Stacking-4
    0.7060    # Stacking-2
])
acc_stds = np.array([
    0.0125, 0.0166, 0.0130, 0.0176, 0.0138, 0.0204, 0.0139
])

f1_means = np.array([
    0.4729,  # Decision Tree
    0.6207,  # Bagging
    0.6761,  # AdaBoost
    0.6316,  # Random Forest
    0.7048,  # SVC
    0.7155,  # Stacking-4
    0.7063   # Stacking-2
])
f1_stds = np.array([
    0.0119, 0.0175, 0.0136, 0.0181, 0.0139, 0.0207, 0.0142
])

# ----------------------------------------------------------
# 2. Bar-plot: Accuracy усіх моделей
# ----------------------------------------------------------
plt.figure(figsize=(8, 5))
x = np.arange(len(model_names))
plt.bar(x, acc_means, yerr=acc_stds, capsize=3)
plt.xticks(x, model_names, rotation=45, ha="right")
plt.ylabel("Accuracy")
plt.title("Mean Accuracy of Each Model (±1 SD)")
plt.tight_layout()
plt.show()

# ----------------------------------------------------------
# 3. Bar-plot: SVC vs 4-model stacking (з анотацією приросту)
# ----------------------------------------------------------
model_names_svc_4 = ["SVC", "Stacking (4-model)"]
acc_means_svc_4   = np.array([0.7035, 0.7152])
acc_stds_svc_4    = np.array([0.0138, 0.0204])

plt.figure(figsize=(5, 4))
x2 = np.arange(len(model_names_svc_4))
plt.bar(x2, acc_means_svc_4, yerr=acc_stds_svc_4, capsize=3)
plt.xticks(x2, model_names_svc_4)
plt.ylabel("Accuracy")
plt.title("SVC vs. 4-model Stacking")
plt.ylim(0.68, 0.73)

improvement = acc_means_svc_4[1] - acc_means_svc_4[0]
plt.text(1, acc_means_svc_4[1] + 0.001,
         f"+{improvement*100:.1f}%", ha="center",
         va="bottom", fontsize=10, fontweight="bold", color="green")
plt.plot([0, 1], acc_means_svc_4, "k--", linewidth=1)
plt.tight_layout()
plt.show()

# ----------------------------------------------------------
# 4. Лінійний графік: F1-score усіх моделей
# ----------------------------------------------------------
plt.figure(figsize=(8, 5))
x3 = np.arange(len(model_names))
plt.plot(x3, f1_means, "o-", linewidth=2, markersize=8)
plt.xticks(x3, model_names, rotation=45, ha="right")
plt.ylabel("F1-Score")
plt.title("Mean F1-Score of Each Model")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Збір точності по фолдах для кожної моделі
acc_data = []
for model_name, fold_metrics in all_metrics.items():
    row = [m["accuracy"] for m in fold_metrics]
    row += [np.mean(row), np.std(row)]
    acc_data.append(row)

fold_labels = [f"Fold {i+1}" for i in range(len(next(iter(all_metrics.values()))))]
columns = fold_labels + ["Mean", "Std"]

acc_df = pd.DataFrame(acc_data, index=all_metrics.keys(), columns=columns)
acc_df = acc_df.round(4)  

display(acc_df)

In [ ]:
import numpy as np
from scipy.stats import ttest_rel

acc_svc     = [m['accuracy'] for m in all_metrics['SVC']]
acc_stack4  = [m['accuracy'] for m in all_metrics['Stacking (4-model)']]

# парний t-тест
t_stat, p_val = ttest_rel(acc_stack4, acc_svc)

alpha = 0.05                                # рівень значущості
mean_svc, mean_stack = np.mean(acc_svc), np.mean(acc_stack4)

if p_val < alpha:
    if mean_stack > mean_svc:
        verdict = "PASS ✅  (Stack 4-model значуще кращий за SVC)"
    else:
        verdict = "FAIL ❌  (SVC значуще кращий за Stack 4-model)"
else:
    verdict = "INCONCLUSIVE ⚠️  (різниця не є статистично значущою)"

print(f"Paired t-test Stack4 vs SVC →  t = {t_stat:.3f},  p = {p_val:.4f}")
print(f"Mean accuracy:  Stack4 = {mean_stack:.4f} | SVC = {mean_svc:.4f}")
print(verdict)

In [ ]:
import pandas as pd
import numpy as np
import os, joblib, ast, warnings
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm  # <-- додано tqdm

warnings.filterwarnings("ignore")

BASE_DIR      = r"C:\Users\Climclaff\Downloads\fma\fma_metadata"
tracks_path   = os.path.join(BASE_DIR, "tracks.csv")
features_path = os.path.join(BASE_DIR, "features.csv")

tracks_df   = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
features_df = pd.read_csv(features_path, index_col=0, header=[0, 1, 2])

tracks_df[('track', 'genre_top')] = tracks_df[('track', 'genre_top')].astype('category')

mask_small   = tracks_df[('set', 'subset')] == 'small'
mask_genre   = ~tracks_df[('track', 'genre_top')].isin(['Pop', 'Folk'])
mask_final   = mask_small & mask_genre

all_cols = ["_".join(col).lower() for col in features_df.columns]
features_df.columns = all_cols

groups_keep = ["chroma_cqt", "chroma_stft", "mfcc", "rmse",
               "spectral_centroid", "spectral_contrast", "tonnetz", "zcr"]

sel_cols = [c for c in all_cols if any(g in c for g in groups_keep)]
X = features_df.loc[mask_final, sel_cols]
y = tracks_df.loc[mask_final, ('track', 'genre_top')]

# Вибірка для швидкої інтерпретації
sample_frac = 0.15
X_sample_full = X.sample(frac=sample_frac, random_state=99)
X_sample = X_sample_full
y_sample = y.loc[X_sample_full.index]


LOAD_DIR = r"E:\ML_Models"
bagging_model       = joblib.load(os.path.join(LOAD_DIR, "Bagging.pkl"))
adaboost_model      = joblib.load(os.path.join(LOAD_DIR, "AdaBoost.pkl"))
random_forest_model = joblib.load(os.path.join(LOAD_DIR, "RandomForest.pkl"))
svc_model           = joblib.load(os.path.join(LOAD_DIR, "SVC.pkl"))

models_dict = {
    "Bagging"      : bagging_model,
    "AdaBoost"     : adaboost_model,
    "RandomForest" : random_forest_model,
    "SVC"          : svc_model,
}

def tree_importances(model, feature_names):
    if hasattr(model, "feature_importances_"):
        return pd.Series(model.feature_importances_, index=feature_names)

    if hasattr(model, "estimators_"):
        importance_dict = {}

        for est in model.estimators_:
            if hasattr(est, "feature_importances_"):
                used_features = est.feature_names_in_ if hasattr(est, "feature_names_in_") else feature_names
                for feat, val in zip(used_features, est.feature_importances_):
                    importance_dict[feat] = importance_dict.get(feat, 0) + val

        for k in importance_dict:
            importance_dict[k] /= len(model.estimators_)

        return pd.Series(importance_dict).reindex(feature_names).fillna(0)

    raise AttributeError("Model does not support feature importance.")


importance_frames = {}

for name, mdl in tqdm(models_dict.items(), desc="Обчислення важливості ознак"):
    if name == "SVC":
        perm = permutation_importance(mdl, X_sample.values, y_sample.values,
                                      n_repeats=1, random_state=99, n_jobs=-1)
        s = pd.Series(perm.importances_mean, index=X_sample.columns)
    else:
        s = tree_importances(mdl, X.columns)
    s = s.sort_values(ascending=False).head(15)
    importance_frames[name] = s

# -------------------------------------------------------------------
# 6.  Виведення результатів
# -------------------------------------------------------------------
for name, imp in importance_frames.items():
    print(f"\n=== TOP-15 FEATURE IMPORTANCE — {name} ===")
    print(imp.round(4))

In [ ]:
import os, ast, time, numpy as np, pandas as pd
from collections import OrderedDict
from sklearn.tree import DecisionTreeClassifier
model_factories = OrderedDict([
    ("DecisionTree",      lambda: DecisionTreeClassifier(max_depth=11,
                                                         min_samples_split=12,
                                                         random_state=213)),
    ("Bagging",           create_bagging),
    ("AdaBoost",          create_adaboost),
    ("RandomForest",      create_random_forest),
    ("SVC",               create_svc),
    ("Stack_4_model",     None),   # буде треновано нижче окремо
    ("Stack_2_model",     None),
])

# ---------------------------
# D. Вимір часу тренування stand-alone моделей
# ---------------------------
train_times = {}
for name, factory in model_factories.items():
    if factory is None:         # стекінгові моделі натренуємо окремо
        continue
    t0 = time.perf_counter()
    _ = factory().fit(X_all, y_all)
    train_times[name] = time.perf_counter() - t0
    print(f"{name:<13}  trained in  {train_times[name]:.2f}  sec")

